Autores: ....

### EDA

#### Normalización de datos y validación

* Utilizamos info() para revisar la estructura y los tipos de datos del conjunto.
    * Los detalles concretos con shape, columns o dtypes.
* Clasificación de región en Regiones y Ciudades
* Explorar datos, fechas, nulos, columnas innecesarias
* Discusión sobre qué hacer con los datos de semanas que faltan (interpolante)

##### Mostrar la estructura de datos del dataset avocados

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.patches as mpatches
import avocado_manager as av
av.init()
df_cp = av.df("df_cp")
print(df_cp.info())

df_cp_cleaned = df_cp
df_cp_cleaned['Suma Volums'] = df_cp_cleaned['Volume_Hass_S' ]+ df_cp_cleaned['Volume_Hass_L']+ df_cp_cleaned['Volume_Hass_XL' ]
df_cp_cleaned['Variacio'] = (df_cp_cleaned['Total Volume'] - df_cp_cleaned['Suma Volums'])*100/df_cp_cleaned['Total Volume']
av.add(df_cp_cleaned,'df_cleaned')

In [ ]:
data_shape = df_cp.shape
print(f"\nForma del DataFrame (filas, columnas): {data_shape}")

column_names = df_cp.columns
print(f"\nNombres de Columnas: {column_names}")
 
data_types = df_cp.dtypes
print(f"\nTipos de datos de cada columna:\n{data_types}")

##### Nuestra clasifiacion de clases de regiones

In [ ]:
print("Nuestra clasifiacion de clases de regiones:",pd.unique(df_cp['region_class']))
display(df_cp.head(5))

##### Mostrar las Regions top 10 por Total Volume

In [ ]:
df_totales = df_cp.groupby('region')['Total Volume'].sum().reset_index()
display(df_totales.nlargest(10,'Total Volume').sort_values(by='Total Volume', ascending = False))
#df_largest = av.df("region_largest")
#display(df_largest)

##### Identificar fechas faltantes por cada region

In [ ]:
df_count = df_cp.groupby(['Date', 'region']).size().reset_index(name='Total')
df_pivot = df_count.pivot(index='Date', columns='region', values='Total').fillna(0)
diferencias = df_pivot.apply(lambda x: x.nunique() > 1, axis=0)

regiones_con_diferencias = diferencias[diferencias].index.tolist()
df_pivot[regiones_con_diferencias].plot(kind='line', marker='.',figsize=(10, 3))
plt.title('Número de filas(registros) por Región a lo largo del tiempo')
plt.xlabel('Fecha')
plt.ylabel('Total de Registros')
plt.legend(title='Regiones:')
plt.grid(); plt.tight_layout() #plt.xticks(rotation=90)
plt.show()

##### Análisis exploratorio para entender la estructura del conjunto, incluyendo el número de filas y columnas, tipos de datos y valores faltantes. 

* Imprime la cantidad de valores faltantes por columna utilizando isnull().

In [ ]:
print(f"Cantidad de nulls: {df_cp.isnull().sum()}")
#df.isnull
#missing_values = pd.isnull(df)

# Resumen Estadístico
data_summary = df_cp.describe()
print("\nResumen Estadístico:")
print(data_summary)

# series de tiempo
print("\nseries de tiempo:")
df_cp['Date'] = pd.to_datetime(df_cp['Date'])
ventas_mensual = df_cp.groupby(df_cp['Date'].dt.to_period("M"))['Total Volume'].sum().reset_index()
display(ventas_mensual)

#### Visión global de datos

* Analisis de Series Temporales y Ruido asociado
* Precios promedio calibre 
* Precios promedio por bolsa
* Proporcion por tipo de bolsa
* Separar avocados convencional y organics
* Comparación precios promedio convencional y organico
* Representación de ventas totales sobre precio promedio ( por regiones o no )

##### Analisis de Series Temporales y Ruido asociado
Observaciones:

* TODO: Perido de 52 semanas para ver la evolucion a la lo largo de los 3 años ?

In [ ]:
print(f"Ragos de fechas: mínima: {df_cp.Date.min()} máxima: {df_cp.Date.max()}")

df_grouped = df_cp.groupby('Date')['AveragePrice'].mean()
df_cp_conventional = df_cp[df_cp['type'] =='conventional']#[['Date', 'AveragePrice']]
df_grouped = df_cp_conventional.groupby('Date')['AveragePrice'].mean()

# Descomposicio de la serie de tiemps: 39 mesos si considerem que tenim del 1-1-2015 fins al 25-3-2018
df_decomp = seasonal_decompose(df_grouped, model='additive', period=(int)(52*1)) #maxim: 84

plt.figure(figsize=(10, 2))
plt.plot(df_decomp.observed, label='avocado convencional')
plt.legend(loc='best');plt.grid(True)
#plt.title('Tendencia')
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 2))
plt.plot(df_decomp.trend, color='red', label='Tendencia mensual(avocado convencional)')
plt.legend(loc='best');plt.grid(True)
#plt.title('Tendencia')
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 2))
plt.plot(df_decomp.seasonal, label='Estacionalidad (avocado convencional)')
plt.legend(loc='best');plt.grid(True)
#plt.title('Estacionalidad')
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 2))
plt.plot(df_decomp.resid, color='green', label='Residuo (avocado convencional)')
plt.legend(loc='best');plt.grid(True)
#plt.title('Residuo')
plt.tight_layout()
plt.show()

In [ ]:
#print("Organic")
df_grouped = df_cp.groupby('Date')['AveragePrice'].mean()
df_cp_organic = df_cp[df_cp['type'] =='organic']
df_grouped = df_cp_conventional.groupby('Date')['AveragePrice'].mean()
# Descomposicio de la serie de tiemps: 39 mesos si considerem que tenim del 1-1-2015 fins al 25-3-2018
df_decomp = seasonal_decompose(df_grouped, model='additive', period=(int)(52*1)) #maxim: 84
plt.figure(figsize=(10, 8))

plt.subplot(411)
plt.plot(df_grouped, label='avocado Organic')
plt.legend(loc='best');plt.grid(True)
plt.subplot(412)
plt.plot(df_decomp.trend, color='red', label='Tendencia (avocado Organic)')
plt.legend(loc='best');plt.grid(True)
plt.subplot(413)
plt.plot(df_decomp.seasonal,label='Estacionalidad (avocado Organic)')
plt.legend(loc='best');plt.grid(True)
plt.subplot(414)
plt.plot(df_decomp.resid, color='green',label='Residuales (avocado Organic)')
plt.legend(loc='best');plt.grid(True)
plt.tight_layout()
plt.show()

##### Precios promedio calibre 

##### Precios promedio por bolsa

##### Proporcion por tipo de bolsa

In [ ]:
df_baggy = df_cp.copy()
region_largest= df_cp.groupby('region')['Total Volume'].sum().nlargest(10).index
df_baggy = df_baggy[df_baggy['region'].isin(region_largest)]

df_baggy['Small Bags ratio'] = df_baggy['Small Bags'] / df_baggy['Total Bags']
df_baggy['Large Bags ratio'] = df_baggy['Large Bags'] / df_baggy['Total Bags']
df_baggy['XLarge Bags ratio'] = df_baggy['XLarge Bags'] / df_baggy['Total Bags']

fig, ax= plt.subplots(figsize=(12,3))
ratio_bag=df_baggy.groupby('region')[['Small Bags ratio','Large Bags ratio', 'XLarge Bags ratio']].mean()
ratio_bag.plot(kind='bar', color=['green', 'orange', 'red'],ax=ax )

In [ ]:
corr_df= df_baggy[['AveragePrice', 'Small Bags', 'Large Bags', 'XLarge Bags']]
# Calcular la matriz de correlación
corr_matrix = corr_df.corr()

# Visualizar la matriz de correlación
plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de Correlación (nlargest(10))')
plt.show()

# Identificar columnas con correlación alta (umbral = 0.8)
threshold = 0.8
to_drop = []
for column in corr_matrix.columns:
    if any((corr_matrix[column].abs() > threshold) & (corr_matrix.index != column)):
        to_drop.append(column)
        
print(f"Variables altamente correlacionadas con otras: {to_drop}")

In [ ]:
df_baggy = df_cp.copy()
region_largest= df_cp.groupby('region')['Total Volume'].sum().nlargest(10).index
df_baggy = df_baggy[df_baggy['region'].isin(region_largest)]

df_baggy['Small Bags ratio'] = df_baggy['Small Bags'] / df_baggy['Total Bags']
df_baggy['Large Bags ratio'] = df_baggy['Large Bags'] / df_baggy['Total Bags']
df_baggy['XLarge Bags ratio'] = df_baggy['XLarge Bags'] / df_baggy['Total Bags']

df_baggy=df_baggy[df_baggy['region']=='LosAngeles']

corr_df= df_baggy[['AveragePrice', 'Small Bags', 'Large Bags', 'XLarge Bags']]
# Calcular la matriz de correlación
corr_matrix = corr_df.corr()

# Visualizar la matriz de correlación
plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de Correlación (LosAngeles)')
plt.show()

# Identificar columnas con correlación alta (umbral = 0.8)
threshold = 0.8
to_drop = []
for column in corr_matrix.columns:
    if any((corr_matrix[column].abs() > threshold) & (corr_matrix.index != column)):
        to_drop.append(column)
        
print(f"Variables altamente correlacionadas con otras: {to_drop}")

##### Precio medio y volumen total
Observaciones:
* TODO: Estamos mostrado los de tipo convencional. Que es el grupo de puntos de la parte superior ?

In [ ]:
df_cp_tmp = df_cp[df_cp['type']=='conventional']

fig, ax = plt.subplots(1, 2, figsize=(18, 6))
ax[0].scatter(df_cp_tmp['AveragePrice'], df_cp_tmp['Total Volume'], alpha=0.5)
ax[0].set_title('Relación entre Precio medio y volumen total de no organicos')
ax[0].set_xlabel('Averag ePrice')
ax[0].set_ylabel('Total Volume')
ax[0].grid()

ax[1].scatter(df_cp_tmp['AveragePrice'], df_cp_tmp['Total Volume'], alpha=0.5)
ax[1].set_title('Relación entre Precio medio y volumen total de no organicos (Log)')
ax[1].set_xlabel('Averag ePrice')
ax[1].set_ylabel('Total Volume')
ax[1].grid()
ax[1].set_yscale('log')

plt.tight_layout()
# df_cp.groupby(by=['region']).count()

##### Separar avocados convencional y organics
Observaciones:
* TODO: Aqui es veu clarament que el alvocat més relevant es el convencional per ordres de magnitud, per ho que seria relevant fer algún estudi del subset eliminant els organics o estudian-los per separat.
* Addicionalment, l'any 2018 no esta complert, pot afectar a les dades.

In [ ]:
grouped = df_cp.groupby(['year', 'type'])['Total Volume'].sum()
fig, ax= plt.subplots(figsize=(12,3))
grouped.plot(kind='bar', color=['grey', 'green'])
plt.xlabel('Año y tipo')
plt.ylabel('Total Volume')
plt.xticks(rotation=45, ha='right')
plt.show()

In [ ]:
df_grouped = df_cp.groupby('Date')['AveragePrice'].mean()

plt.figure(figsize=(12, 3))
plt.plot(df_grouped, label ='Precio Medio')
plt.legend(loc='best')
plt.xlabel('Tiempo')
plt.ylabel('Precio medio')
plt.title('Precio medio en el tiempo')
plt.grid(True); plt.xticks(rotation=0)
plt.show()

df_grouped = df_cp.groupby(pd.Grouper(key='Date', freq='M'))['AveragePrice'].mean()
plt.figure(figsize=(12,3))
plt.plot(df_grouped, label='Precio medio mensual') #plt.plot(df_grouped.index, df_grouped.values)
plt.legend(loc='best')
plt.xlabel('Fecha')
plt.ylabel('Precio medio')
plt.title('Precio medio mensual')
plt.grid(True); plt.xticks(rotation=0)
plt.show()

df_cp_conventional = df_cp[df_cp['type'] =='conventional'] #[['Date', 'AveragePrice']]
df_grouped_conv = df_cp_conventional.groupby('Date')['AveragePrice'].mean()

df_cp_organic = df_cp[df_cp['type'] =='organic'] #[['Date', 'AveragePrice']]
df_grouped_orga = df_cp_organic.groupby('Date')['AveragePrice'].mean()

plt.figure(figsize=(12, 3))
plt.plot(df_grouped_conv, label ='Precio Medio convencional', color ='grey')
plt.plot(df_grouped_orga, label ='Precio Medio organic', color ='green')
plt.legend(loc='best')
plt.xlabel('Tiempo')
plt.ylabel('Precio medio')
plt.title('Precio medio en el tiempo segun tipo alvocado')
plt.grid(True); plt.xticks(rotation=0)
plt.show()

#df_grouped

##### Comparación precios promedio convencional y organico
Observaciones:
* TODO: Aqui veiem que existeixen unes regions mes relevants que altres. Caldria fer una separació?
* TODO: Excloure TotalUS ??

In [ ]:
df_subset = df_cp
#df_subset = df_cp[df_cp['region']!= 'TotalUS']
pd.unique(df_subset['region'])

grouped = df_subset.groupby(['region', 'type'])['Total Volume'].sum()

fig, ax= plt.subplots(figsize=(20,4))
grouped.plot(kind='bar', ax=ax, color = ['gray', 'green'] )
# plt.ylim(0, 0.3*10**9)
plt.grid()

In [ ]:
price_group = df_subset.groupby(['region', 'type'])['AveragePrice'].mean()#.nlargest(10)

fig, ax= plt.subplots(figsize=(20,4))
price_group.plot(kind='bar', ax=ax, color = ['gray', 'green'] )
# plt.ylim(0, 0.3*10**9)
plt.grid()

Observaciones:
* TODO: Aquest gràfic es espectacular, y ens permet veure que sempre són els alvocats orgànics els que costen més que la mitjana.

In [ ]:
avocados_region_mean = df_subset.groupby(['region','type'])['AveragePrice'].mean()#.nlargest(6)

total_mean = df_subset['AveragePrice'].mean()
fig, ax= plt.subplots(figsize=(20,4))
avocados_relative_mean = (avocados_region_mean - total_mean)*100/total_mean
avocados_relative_mean.plot(kind = 'bar', ylabel= '% Price deviation', color = ['slategray', 'forestgreen'])
plt.grid()

##### TODO: Convencional vs Organic
Observaciones:
* TODO: 
  * Si mirem la proporcio, en general es manté constant. No hi ha ningun interes per a tornar a ho natural a les gran ciutats o extensions.
  * Si acas hi ha més divergencia en extensions menors.

In [ ]:
grouped = df_subset.groupby(['region', 'type'])['Total Volume'].sum()
unstacked_type= grouped.unstack()
unstacked_type['Proportion'] = unstacked_type['conventional']/unstacked_type['organic']

fig, ax= plt.subplots(figsize=(12,4))
x_values= unstacked_type['conventional']
y_values= unstacked_type['Proportion']
ax.scatter(x=x_values, y= y_values )
# plt.ylim(0, 0.3*10**9)
plt.grid()

##### Ventas en el tiempo y separación entre convencional / orgánico

In [ ]:
df_date_volume = df_cp[['Date', 'Total Volume']]
#df_cp_organic = df_cp[df_cp['type'] =='organic'] #[['Date', 'AveragePrice']]
#df_date_volume
df_grouped_conventional = df_date_volume[df_cp['type'] =='conventional'].groupby('Date').sum('Total Volume')
df_grouped_organic = df_date_volume[df_cp['type'] =='organic'].groupby('Date').sum('Total Volume')

# Graficar
plt.figure(figsize=(12,3))
plt.title('Volumen total de ventas a lo targo del tiempo')
plt.plot(df_grouped_conventional, label='Volumen total ventas (conventional)')
plt.xlabel('Fecha')
plt.ylabel('Volumen ventas')
plt.legend(loc='best'); plt.grid(True)
plt.plot(df_grouped_organic, label='Volumen total ventas (organic)')
plt.xlabel('Fecha')
plt.ylabel('Volumen ventas')
plt.legend(loc='best'); plt.grid(True)
plt.show()

plt.figure(figsize=(12,3))
plt.title('Volumen total de ventas a lo targo del tiempo - escala logaritmica')
plt.plot(df_grouped_conventional, label='Volumen total ventas (conventional) escala logaritmica')
plt.xlabel('Fecha')
plt.ylabel('Volumen ventas')
plt.yscale('log')
plt.legend(loc='best'); plt.grid(True)
plt.plot(df_grouped_organic, label='Volumen total ventas (organic) escala logaritmica')
plt.xlabel('Fecha')
plt.ylabel('Volumen ventas')
plt.yscale('log')
plt.legend(loc='best'); plt.grid(True)
plt.show()

##### Análisis de Cambios en Precios Anuales:

In [ ]:
df_year_price = df_cp.groupby('year')['AveragePrice'].mean()
df_year_price_conventional = df_cp[df_cp['type'] =='conventional'].groupby('year')['AveragePrice'].mean()
df_year_price_organic = df_cp[df_cp['type'] =='organic'].groupby('year')['AveragePrice'].mean()
display(df_year_price)

# Graficar
plt.figure(figsize=(10, 2)) 
df_year_price.plot.bar(label ='Precio medio', color ='blue', alpha=1) # x='aaa??',y='bbb??',
plt.xlabel('Año')
plt.ylabel('Precio medio')
plt.title('Precio medio anual')
plt.legend(loc='lower right'); plt.grid(True)
plt.xticks(rotation=0)
plt.show()


plt.figure(figsize=(10, 2)) 
df_year_price_organic.plot.bar(label ='Precio medio alvocado organic', color ='green') # x='aaa??',y='bbb??',
df_year_price_conventional.plot.bar(label ='Precio medio alvocado conventional', color ='grey') # x='aaa??',y='bbb??',
#df_year_price.plot.bar(label ='Precio medio', color ='blue', alpha=0.2) # x='aaa??',y='bbb??',
plt.xlabel('Año')
plt.ylabel('Precio medio')
plt.title('Precio medio anual')
plt.legend(loc='lower right'); plt.grid(True)
plt.xticks(rotation=0)
plt.show()

##### Representación de ventas totales sobre precio promedio ( por regiones o no )

In [ ]:
classification_colors = {'City':'green' ,'Region':'red' ,'GreaterRegion':'orange', 'TotalUS': 'blue'}

#df_subset = av.df("df_city_region") 
df_subset = df_cp[df_cp['region']!= 'TotalUS']

df_organic = df_subset[df_subset['type']=='organic']
df_convencionals = df_subset[df_subset['type']=='conventional']

fig, ax= plt.subplots(figsize=(12,4))
x_values = df_convencionals['Total Volume']
y_values = df_convencionals['AveragePrice']
c_values= list(df_convencionals['region_class'].map(classification_colors))
plt.scatter(x= x_values, y= y_values, c=c_values, alpha = .5)
plt.title('Tipo convencionales (log)')
plt.xscale('log')
plt.ylabel('Average price')
plt.xlabel('Market size')
legend_patches = [mpatches.Patch(color=color, label=region) for region, color in classification_colors.items()]
plt.legend(handles=legend_patches, loc='upper right')
plt.grid()

fig, ax= plt.subplots(figsize=(12,4))
x_values = df_organic['Total Volume']
y_values = df_organic['AveragePrice']
c_values= list(df_organic['region_class'].map(classification_colors))
plt.scatter(x= x_values, y= y_values, c=c_values, alpha = .5)
plt.title('Tipo organicos (log)')
plt.xscale('log')
plt.ylabel('Average price')
plt.xlabel('Market size')
legend_patches = [mpatches.Patch(color=color, label=region) for region, color in classification_colors.items()]
plt.legend(handles=legend_patches, loc='upper right')
plt.grid()

In [ ]:
# df_convencionals
index_df = df_cp_cleaned.copy()
#['West', 'California', 'SouthCentral', 'Northeast', 'Southeast']
great_reg_col = {'California' : 'red', 'West':'Orange', 'SouthCentral':'green', 'Northeast':'SkyBlue', 'Southeast':'DarkBlue'}

region_largest= list(index_df.groupby('region')['Total Volume'].sum().nlargest(5).index)
df_regions = df_convencionals[df_convencionals['region'].isin(region_largest)]

In [ ]:
fig, ax= plt.subplots(figsize=(16,4))
y_values = df_regions['Total Volume']
x_values = df_regions['Date']
c_values= list(df_regions['region'].map(great_reg_col))

plt.scatter(x= x_values, y= y_values, c=c_values, alpha = 1)
plt.yscale('log')
plt.title('Total Volume en el tiempo (log)')
plt.ylabel('Total Volume')
#plt.xlabel('Date')
#plt.legend() #loc='lower right')
legend_handles = [plt.Line2D([0], [0], marker='o', color='w', label=region, markerfacecolor=color, markersize=8) for region, color in great_reg_col.items() if region in df_regions['region'].unique()]
plt.legend(handles=legend_handles, loc='best', bbox_to_anchor=(1, 1), ncol=1)
plt.grid()

In [ ]:
fig, ax= plt.subplots(figsize=(16,4))
y_values = df_regions['AveragePrice']
x_values = df_regions['Date']
c_values= list(df_regions['region'].map(great_reg_col))

plt.scatter(x= x_values, y= y_values, c=c_values, alpha = 1)
#plt.yscale('log')
plt.title('Precio medio en el tiempo')
plt.ylabel('AveragePrice')
#plt.xlabel('Date')
legend_handles = [plt.Line2D([0], [0], marker='o', color='w', label=region, markerfacecolor=color, markersize=8) for region, color in great_reg_col.items() if region in df_regions['region'].unique()]
plt.legend(handles=legend_handles, loc='best', bbox_to_anchor=(1, 1), ncol=1)
plt.grid()

#### Subset alvocados convencionals
* Aqui tendriamos que incluir una idea de si todas las regiones hacen un número similar total de pedidos
* Claramente hay sitios donde los aguacates son más baratos, miremos si tiene que ver con otras características

In [ ]:
df_subset = df_cp[df_cp['region']!= 'TotalUS']
df_convencionals = df_subset[df_subset['type']=='conventional']
convencional_region_mean = df_convencionals.groupby('region')['AveragePrice'].mean()#.nlargest(6)
coloring=convencional_region_mean.index.map(av.region_classification).map(av.classification_colors)
total_mean = df_convencionals['AveragePrice'].mean()
fig, ax= plt.subplots(figsize=(20,4))
avocados_relative_mean = ((convencional_region_mean - total_mean)*100/total_mean)
avocados_relative_mean.plot(kind = 'bar', ylabel= '% Price deviation')#, color= coloring)#, color= convencional_region_mean.index.map(classification_colors))
plt.grid()

#### Majors mercats el preu mig tendeix a abaratirse
* Aqui podem veure que a majors mercats el preu mig tendeix a abaratirse. 
* Aqui estem prenent cada regio com equivalentment valida
* cambiar el 2018 no parece afectar los resultados

In [ ]:
convencional_region_mean_total = df_convencionals.groupby('region').agg({'Total Volume':'sum', 'AveragePrice':'mean'})#.nlargest(6)
convencional_region_mean_total['region_class'] = convencional_region_mean_total.index.map(av.region_classification)

x_values = convencional_region_mean_total['Total Volume']
total_mean = convencional_region_mean_total['AveragePrice'].mean()
y_values = (convencional_region_mean_total['AveragePrice'] - total_mean)*100/total_mean
c_values= list(convencional_region_mean_total['region_class'].map(av.classification_colors))

fig, ax= plt.subplots(figsize=(12,6))
plt.scatter(x= x_values, y= y_values, c=c_values)
plt.xscale('log')
plt.ylabel('% Average price deviation')
plt.xlabel('Market size')
legend_patches = [mpatches.Patch(color=color, label=region) for region, color in av.classification_colors.items()]
plt.legend(handles=legend_patches, loc='best', bbox_to_anchor=(1, 1), ncol=1)
plt.grid()

### ANALISIS

#### Separar alvocats convencionals i alvocats orgànics

#### Impacto del precio en las ventas

#### Estacionalidad por región

In [ ]:
get_season = av.get_season()

df_date_price_volume = df_cp[['Date', 'region', 'AveragePrice', 'Total Volume']]
df_date_price_volume = df_date_price_volume.reset_index()
df_date_price_volume['Season'] = df_date_price_volume['Date'].apply(get_season)
# df_date_price_volume

df_grouped_mean = df_date_price_volume.groupby(['Season','region'])['AveragePrice'].mean()
df_grouped_mean = df_grouped_mean.reset_index()

seasons = df_grouped_mean['Season'].unique()
plt.figure(figsize=(20, 8)) 
for season in seasons:
    df_season = df_grouped_mean[df_grouped_mean['Season'] == season]
    plt.plot(df_season['region'], df_season['AveragePrice'], marker='.', label=season)
plt.title('Precio medio por temporada y región')
plt.xlabel('Región')
plt.ylabel('Precio medio')
plt.xticks(rotation=90)
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()
#df_grouped_mean

#### Correlacion volumen total entre de las "grandes regiones" y "regiones" con total US
* TODO: IMPORTANT VOLUM VENTAS GREATERREGIONS == TOTALUS

In [ ]:
df_great_regions = df_cp[(df_cp['region_class']=='Region') | (df_cp['region_class']=='City')]
df_USA = df_cp[df_cp['region']=='TotalUS']
print((df_great_regions['Total Volume'].sum() - df_USA['Total Volume'].sum())*100/df_USA['Total Volume'].sum() )

In [ ]:
index_df= df_cp.copy()
index_df= index_df[index_df['region_class']=='City']

region_largest= list(index_df.groupby('region')['Total Volume'].sum().nlargest(3).index)
# region_largest.pop('TotalUS')

df_greatest_pie = df_cp.groupby('region')['Total Volume'].sum().nlargest(8)   #df_cp[df_cp['region'].isin(region_largest)]
df_greatest_pie.loc['TotalUS'] = 2*df_greatest_pie.loc['TotalUS'] - df_greatest_pie.loc[:].sum()
df_greatest_pie.rename( index={'TotalUS':'Plains'}, inplace=True)
df_greatest_pie.plot.pie(title="Market size", legend=False, \
                   autopct='%1.1f%%', \
                   shadow=True, startangle=0)
#plt.legend(loc='best'); plt.grid(True) # figsize=(12,6)

#### Correlacion de Volumen Total de las ciudades con Volumen de Total US (alvocado conventional)

In [ ]:
prova_= df_cp.copy()
index_df= df_cp.copy()
prova_= prova_[prova_['type']=='conventional']
dictionary_to_regions={}

index_df= index_df[index_df['region_class']=='GreaterRegion']
region_largest= list(index_df.groupby('region')['Total Volume'].sum().nlargest(5).index)
region_largest.append('TotalUS')

for region in region_largest:
    prova_temp = prova_[prova_['region']==region]
    prova_temp=prova_temp.sort_values(by='Date')

    dictionary_to_regions[region] = list(prova_temp['Total Bags'])

df_corr = pd.DataFrame(dictionary_to_regions)
# display(df_corr)
corr_matrix = df_corr.corr()

# Visualizar la matriz de correlación
plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de Correlación')
plt.show()

# Identificar columnas con correlación alta (umbral = 0.8)
threshold = 0.9
to_drop = []
for column in corr_matrix.columns:
    if any((corr_matrix[column].abs() > threshold) & (corr_matrix.index != column)):
        to_drop.append(column)
        
print(f"Variables altamente correlacionadas con otras: {to_drop}")

#### Elasticidad

#### Analisis por cohortes

##### 1. Cohortes Basadas en Precios Promedios Trimestrales:

In [ ]:
df_USA = df_cp_cleaned[df_cp_cleaned['region']=='TotalUS'].copy()
df_USA = df_USA[df_USA['type']=='conventional']#.copy()
df_media = df_USA.groupby([pd.Grouper(key='Date', freq='QS')]).agg({'AveragePrice':'mean', 'Total Volume':'sum'}).reset_index()
df_media = df_USA.groupby(['Date']).agg({'AveragePrice':'mean', 'Total Volume':'sum'}).reset_index()

df_media['Date offset'] = df_media['Date'] - pd.DateOffset(months=5)

# display(df_temp)
#%%
fig, ax1 = plt.subplots(figsize=(12,4))
#plt.title('Estudio economico')
color = 'tab:red'
ax1.set_xlabel('Data')
ax1.set_ylabel('Volum ventas', color=color)
ax1.plot(df_media['Date'], df_media['Total Volume'], color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(color=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
color = 'tab:blue'
ax2.set_ylabel('Average Price (€)', color=color)  # we already handled the x-label with ax1
ax2.plot(df_media['Date'], df_media['AveragePrice'], color=color)
ax2.tick_params(axis='y', labelcolor=color)
# plt.ylim(0, 5)
#ax2.grid(color=color)
fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.xticks(rotation = 90)
plt.show()

##### 2. Cohortes por Región y Fecha:

In [ ]:
df_no_USA = df_cp_cleaned[df_cp_cleaned['region']=='California']
df_media = df_no_USA.groupby(['region', 'year']).agg({'Total Volume':'sum', 'AveragePrice':'mean'})#.reset_index()

df_media['Total Volume'].plot(kind='bar', label = 'Total Volume', color= 'orange')
plt.show()

df_media['AveragePrice'].plot(kind='bar', label = 'Average Price', color= 'green')
plt.show()

##### 3. Análisis de Cohortes en Función del Tipo de Bolsa:

In [ ]:
columnas_bags = ['Total Bags', 'Small Bags', 'Large Bags', 'XLarge Bags']

# df_bag_temp = df_cp_cleaned[df_cp_cleaned['region']=='Philadelphia'].copy()
df_bag_temp = df_cp_cleaned.copy()
df_bag_temp= df_cp_cleaned.groupby('Date')['Total Bags'].sum()

fig, ax= plt.subplots()
for title_bag in columnas_bags: 
    df_bag_temp= df_cp_cleaned.groupby('Date')[title_bag].sum()
    df_bag_temp.plot(kind = 'line', ax=ax, label= title_bag)

plt.yscale('log')
plt.title('Logarithmic')
plt.grid()
plt.legend()
plt.show()

In [ ]:
columnas_bags = ['Total Bags', 'Small Bags', 'Large Bags', 'XLarge Bags']

fig, ax= plt.subplots()
for title_bag in columnas_bags:
    df_bag_temp= df_cp_cleaned.groupby('Date')[title_bag].sum()
    df_bag_temp.plot(kind = 'line', ax=ax, label= title_bag)

# plt.yscale('log')   
plt.grid()
plt.legend()
plt.show()

In [ ]:
columnas_bags = ['Total Bags', 'Small Bags', 'Large Bags', 'XLarge Bags']
fig, ax= plt.subplots()
for title_bag in columnas_bags:
    df_bag_temp= df_cp_cleaned.groupby([pd.Grouper(key='Date', freq='QS')])[title_bag].sum()
    df_bag_temp.plot(kind = 'line', ax=ax, label= title_bag)

# plt.yscale('log')   
plt.grid()
plt.legend()
plt.show()

##### 4. Cohortes de Clientes Basadas en Ventas:

In [ ]:
df_cleaned_great_regions = df_cp_cleaned[df_cp_cleaned['region_class']=='GreaterRegion']
# df_cleaned_great_regions = df_cleaned_great_regions[(df_cleaned_great_regions['Date']<'15-03-2018')&(df_cleaned_great_regions['Date']>'01-01-2018')]
great_regions = pd.unique(df_cleaned_great_regions['region'])

fig, ax= plt.subplots(figsize=(20,8))
for region in great_regions:
    df_temp_region = df_cleaned_great_regions[df_cleaned_great_regions['region'] ==region ]
    df_date_region= df_temp_region.groupby('Date')['Total Volume'].sum()
    df_date_region.plot(kind = 'line', ax=ax, label= region)

# plt.yscale('log')
plt.grid()
plt.legend()
plt.show()

In [ ]:
df_cleaned_great_regions = df_cp_cleaned[df_cp_cleaned['region_class']=='GreaterRegion']
df_cleaned_great_regions = df_cleaned_great_regions[(df_cleaned_great_regions['Date']<'2017-06-15')&(df_cleaned_great_regions['Date']>'2017-04-15')]
great_regions = pd.unique(df_cleaned_great_regions['region'])

fig, ax= plt.subplots(figsize=(20,8))
for region in great_regions:
    df_temp_region = df_cleaned_great_regions[df_cleaned_great_regions['region'] ==region ]
    df_date_region= df_temp_region.groupby('Date')['Total Volume'].sum()
    df_date_region.plot(kind = 'line', ax=ax, label= region)

# plt.yscale('log')
plt.grid()
plt.legend()
plt.show()

##### 5. Evaluación de Retención de Ventas por Cohorte:

In [ ]:
df_cleaned_great_regions = df_cp_cleaned[df_cp_cleaned['region_class']=='GreaterRegion']
# df_cleaned_great_regions = df_cleaned_great_regions[(df_cleaned_great_regions['Date']<'15-03-2018')&(df_cleaned_great_regions['Date']>'01-01-2018')]
great_regions = pd.unique(df_cleaned_great_regions['region'])

fig, ax= plt.subplots(figsize=(20,8))
for region in great_regions:
    df_temp_region = df_cleaned_great_regions[df_cleaned_great_regions['region'] ==region ]
    df_date_region= df_temp_region.groupby([pd.Grouper(key='Date', freq='MS')])['Total Volume'].sum()
    df_date_region.plot(kind = 'line', ax=ax, label= region)

# plt.yscale('log')
plt.grid()
plt.legend()
plt.show()

#### Matriz de correlaciones de todas las variables según  apartado 5.6. En especial, precio medio y relación con diferentes calibres.

In [ ]:
df_corr_gen= df_cp.copy()
df_corr_gen = df_corr_gen[df_corr_gen['region']=='Philadelphia']
# df_corr_gen = df_corr_gen[(df_corr_gen['region_class']== 'City') | (df_corr_gen['region_class']== 'Region')]

corr_df= df_corr_gen[['AveragePrice', 'Total Volume', 'Total Bags' ,'Volume_Hass_S', 'Volume_Hass_L', 'Volume_Hass_XL']]
# Calcular la matriz de correlación
corr_matrix = corr_df.corr()

# Visualizar la matriz de correlación
plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de Correlación (Philadelphia)')
plt.show()

# Identificar columnas con correlación alta (umbral = 0.8)
threshold = 0.8
to_drop = []
for column in corr_matrix.columns:
    if any((corr_matrix[column].abs() > threshold) & (corr_matrix.index != column)):
        to_drop.append(column)
        
print(f"Variables altamente correlacionadas con otras: {to_drop}")

#### Dispersión entre variables claves según 5.2

In [ ]:
df_local = df_cp.copy()
df_local= df_local[df_local['type']=='organic']
df_local= df_local[df_local['region']=='TotalUS']
df_local = df_local[df_local['AveragePrice']==1.0]
display(df_local)

* TODO ?

In [ ]:
# df_local = df_cp.copy()
# df_local= df_local[df_local['type']=='conventional']

# # df_local = df_cp[(df_cp['region']!='TotalUS')&(df_cp['region_class']!='GreaterRegion')].copy()
# df_local_USA=df_local[df_local['region_class']=='TotalUS']

# fig, ax= plt.subplots(figsize=(10,6))

# # sns.scatterplot(data= df_local , x=df_local['Total Volume'], y=df_local['AveragePrice'], alpha= 0.1)
# sns.regplot(data= df_local_USA , x=df_local_USA['Total Volume'], y=df_local_USA['AveragePrice'], order=1)#, line=True)
# plt.xscale('log')
# plt.grid()
# plt.title('Conventional')
# plt.figure()

In [ ]:
df_local = df_cp.copy()
df_local= df_local[df_local['type']=='organic']

# df_local = df_cp[(df_cp['region']!='TotalUS')&(df_cp['region_class']!='GreaterRegion')].copy()
df_local_USA=df_local[df_local['region_class']=='TotalUS']

fig, ax= plt.subplots(figsize=(10,6))

# sns.scatterplot(data= df_local , x=df_local['Total Volume'], y=df_local['AveragePrice'], alpha= 0.1)
sns.regplot(data= df_local_USA , x=df_local_USA['Total Volume'], y=df_local_USA['AveragePrice'], order=1)#, line=True)
plt.xscale('log')
plt.grid()
plt.title('TotalUS')
plt.figure()

In [ ]:
df_local = df_cp.copy()
df_local= df_local[df_local['type']=='organic']

# df_local = df_cp[(df_cp['region']!='TotalUS')&(df_cp['region_class']!='GreaterRegion')].copy()
df_local_USA=df_local[df_local['region_class']=='TotalUS']
df_local_USA=df_local_USA[df_local_USA['AveragePrice']>1.1]

fig, ax= plt.subplots(figsize=(10,6))

# sns.scatterplot(data= df_local , x=df_local['Total Volume'], y=df_local['AveragePrice'], alpha= 0.1)
sns.regplot(data= df_local_USA , x=df_local_USA['Total Volume'], y=df_local_USA['AveragePrice'], order=1)#, line=True)
plt.xscale('log')
plt.grid()
plt.title('TotalUS Organic')
plt.figure()

# df_local = df_cp[(df_cp['region']!='TotalUS')&(df_cp['region_class']!='GreaterRegion')].copy()
df_local_region=df_local[df_local['region_class']=='City']

fig, ax= plt.subplots(figsize=(10,6))

# sns.scatterplot(data= df_local , x=df_local['Total Volume'], y=df_local['AveragePrice'], alpha= 0.1)
sns.regplot(data= df_local_region , x=df_local_region['Total Volume'], y=df_local_region['AveragePrice'], order=1)#, line=True)
plt.xscale('log')
plt.title('City Organic')
plt.grid()
plt.figure()

In [ ]:
df_local = df_cp.copy()
df_local= df_local[df_local['type']=='organic']

# df_local = df_cp[(df_cp['region']!='TotalUS')&(df_cp['region_class']!='GreaterRegion')].copy()
df_local_USA=df_local[df_local['region_class']=='TotalUS']

fig, ax= plt.subplots(figsize=(10,6))

# sns.scatterplot(data= df_local , x=df_local['Total Volume'], y=df_local['AveragePrice'], alpha= 0.1)
sns.regplot(data= df_local_USA , x=df_local_USA['Total Volume'], y=df_local_USA['AveragePrice'], order=1)#, line=True)
plt.xscale('log')
plt.grid()
plt.title('TotalUS')
plt.figure()

# df_local = df_cp[(df_cp['region']!='TotalUS')&(df_cp['region_class']!='GreaterRegion')].copy()
df_local_region=df_local[df_local['region_class']=='City']

fig, ax= plt.subplots(figsize=(10,6))

# sns.scatterplot(data= df_local , x=df_local['Total Volume'], y=df_local['AveragePrice'], alpha= 0.1)
sns.regplot(data= df_local_region , x=df_local_region['Total Volume'], y=df_local_region['AveragePrice'], order=1)#, line=True)
plt.xscale('log')
plt.title('City')
plt.grid()
plt.figure()

 ### TODO

In [ ]:
#df_cp_cleaned = av.df("df_cleaned")
prova_= df_cp.copy()
index_df= df_cp.copy()
prova_= prova_[prova_['type']=='conventional']

#great_dates= df_cp_cleaned.copy()
#great_dates= great_dates[great_dates['type']=='conventional']

index_df= df_cp.copy()
index_df= index_df[index_df['region_class']=='GreaterRegion']

region_largest= list(index_df.groupby('region')['Total Volume'].sum().nlargest(5).index)
#region_largest#.append('TotalUS')
fig, ax= plt.subplots(figsize=(20,6))
for region in region_largest:
    prova_temp = prova_[prova_['region']==region]
    prova_temp=prova_temp.sort_values(by='Date')
    prova_temp.plot(x= 'Date', y= 'AveragePrice', ax=ax, label= region)

plt.grid()
plt.show()

In [ ]:
prova_= df_cp.copy()
index_df= df_cp.copy()
prova_= prova_[prova_['type']=='conventional']

index_df= index_df[index_df['region_class']=='GreaterRegion']
region_largest= list(index_df.groupby('region')['Total Volume'].sum().nlargest(5).index)
# region_largest.append('TotalUS')

fig, ax= plt.subplots(figsize=(20,4))
for region in region_largest:
    prova_temp = prova_[prova_['region']==region]
    prova_temp=prova_temp.sort_values(by='Date')
    prova_temp.plot(x= 'Date', y= 'Total Volume', ax=ax, label= region)
plt.legend(loc='best'); plt.grid(True)
plt.show()

In [ ]:
df_avocado_size = df_cp.copy()
df_avocado_size['Volume_Hass_S ratio'] = df_avocado_size['Volume_Hass_S'] / df_avocado_size['Total Volume']
df_avocado_size['Volume_Hass_L ratio'] = df_avocado_size['Volume_Hass_L'] / df_avocado_size['Total Volume']
df_avocado_size['Volume_Hass_XL ratio'] = df_avocado_size['Volume_Hass_XL'] / df_avocado_size['Total Volume']

fig, ax= plt.subplots(figsize=(20,8))
ratio_bag=df_avocado_size.groupby('region')[['Volume_Hass_S ratio', 'Volume_Hass_L ratio', 'Volume_Hass_XL ratio']].mean()
ratio_bag.plot(kind='bar', color=['green', 'orange', 'red'],ax=ax )

In [ ]:
corr_df= df_avocado_size[['AveragePrice', 'Volume_Hass_S ratio', 'Volume_Hass_L ratio', 'Volume_Hass_XL ratio']]
# Calcular la matriz de correlación
corr_matrix = corr_df.corr()

# Visualizar la matriz de correlación
plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de Correlación')
plt.show()

# Identificar columnas con correlación alta (umbral = 0.8)
threshold = 0.8
to_drop = []
for column in corr_matrix.columns:
    if any((corr_matrix[column].abs() > threshold) & (corr_matrix.index != column)):
        to_drop.append(column)
        
print(f"Variables altamente correlacionadas con otras: {to_drop}")

In [ ]:
corr_df= df_cp_cleaned[['Total Volume', 'Volume_Hass_S', 'Volume_Hass_L', 'Volume_Hass_XL']]
# Calcular la matriz de correlación
corr_matrix = corr_df.corr()

# Visualizar la matriz de correlación
plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de Correlación')
plt.show()

# Identificar columnas con correlación alta (umbral = 0.8)
threshold = 0.8
to_drop = []
for column in corr_matrix.columns:
    if any((corr_matrix[column].abs() > threshold) & (corr_matrix.index != column)):
        to_drop.append(column)
        
print(f"Variables altamente correlacionadas con otras: {to_drop}")

#### TODO

In [ ]:
#df_cp_cleaned['Suma Volums'] = df_cp_cleaned['Volume_Hass_S' ]+ df_cp_cleaned['Volume_Hass_L']+ df_cp_cleaned['Volume_Hass_XL' ]
df_cp_cleaned['Variacio'] = (df_cp_cleaned['Total Volume'] - df_cp_cleaned['Suma Volums'])*100/df_cp_cleaned['Total Volume']
smallest_regions= df_cp_cleaned[df_cp_cleaned['region'].isin(region_largest) ]
small_group = df_cp_cleaned.groupby('region')['Variacio'].mean().sort_values()
small_group

In [ ]:
grouper = df_cp_cleaned.groupby('region')['Total Volume'].sum().nlargest(10).index

In [ ]:
bags_df = df_cp_cleaned.copy()
bags_df['Suma Bags'] = bags_df['Small Bags']+ bags_df['Large Bags']+ bags_df['XLarge Bags']
bags_df['Variacio Bags'] = (bags_df['Total Bags'] - bags_df['Suma Bags'])*100/bags_df['Total Bags']
fig, ax= plt.subplots(figsize=(6,3))
bags_df['Variacio Bags'].hist(bins=100)

In [ ]:
df_cp.head()
df_cp.info()
df_cp.describe()

* Aqui podem veure que a majors mercats el preu mig tendeix a abaratirse. Aqui estem prenent cada regio com equivalentment valida
* cambiar el 2018 no parece afectar los resultados

* TODO: Xavi el grafic es veu diferent respecte al que tens a "Xavi/0_data_set_Xavi.ipynb"

In [ ]:
convencional_region_mean_total = df_cp_cleaned.groupby(['region','type']).agg({'Total Volume':'sum', 'AveragePrice':'mean'})#.nlargest(6)

for region in pd.unique(df_cp_cleaned['region']):
    convencional_region_mean_total.loc[(region, 'organic'),'Total Volume'] = convencional_region_mean_total.loc[(region, 'conventional'),'Total Volume']

new_convencional_region_mean_total= convencional_region_mean_total.reset_index(level='type') 

type_coloring= {'organic':'green', 'conventional':'gray'}

# display(new_df)
fig, ax= plt.subplots(figsize=(20,4))
x_values = new_convencional_region_mean_total['Total Volume']
total_mean = new_convencional_region_mean_total['AveragePrice'].mean()
y_values = (new_convencional_region_mean_total['AveragePrice'] - total_mean)*100/total_mean
c_values= new_convencional_region_mean_total['type'].map(type_coloring)

plt.scatter(x= x_values, y= y_values, c=c_values)
plt.ylabel('% Average price deviation')
plt.xlabel('Market size')
plt.grid()

In [ ]:
convencional_region_mean_total = df_convencionals.groupby('region').agg({'Total Volume':'mean', 'AveragePrice':'mean'})#.nlargest(6)
convencional_region_mean_total['region_class'] = convencional_region_mean_total.index.map(av.region_classification)

classification_colors = {'City':'green' ,'Region':'yellow' ,'GreaterRegion':'orange', 'State':'red'}


fig, ax= plt.subplots(figsize=(20,4))
x_values = convencional_region_mean_total['Total Volume']
total_mean = df_convencionals['AveragePrice'].mean()
y_values = (convencional_region_mean_total['AveragePrice'] - total_mean)*100/total_mean
c_values= convencional_region_mean_total['region_class'].map(classification_colors)


plt.scatter(x= x_values, y= y_values)
plt.ylabel('% Average price deviation')
plt.xlabel('Market size')
plt.grid()

In [ ]:
# Aqui podem veure que a majors mercats el preu mig tendeix a abaratirse. Aqui estem prenent cada regio com equivalentment valida
# cambiar el 2018 no parece afectar los resultados
convencional_df = df_cp_cleaned[df_cp_cleaned['type']=='conventional']
organic_df = df_cp_cleaned[df_cp_cleaned['type']=='organic']

convencional_region_mean_total = convencional_df.groupby('region').agg({'Total Volume':'sum', 'AveragePrice':'mean'})#.nlargest(6)
organic_region_mean_total = organic_df.groupby('region').agg({'AveragePrice':'mean'})#.nlargest(6)

convencional_region_mean_total['Price difference'] = organic_region_mean_total['AveragePrice'] - convencional_region_mean_total['AveragePrice'] 

# display(new_df)
fig, ax= plt.subplots(figsize=(20,4))
x_values = convencional_region_mean_total['Total Volume']
# total_mean = new_convencional_region_mean_total['AveragePrice'].mean()
y_values = convencional_region_mean_total['Price difference'] #(new_convencional_region_mean_total['AveragePrice'] - total_mean)*100/total_mean
# c_values= convencional_region_mean_total['type'].map(type_coloring)

plt.scatter(x= x_values, y= y_values)#, c=c_values)
plt.ylabel('Price difference organic vs conventional')
plt.xlabel('Market size')
plt.grid()

### REGRESIONES Y PROYECCIONES

#### Creación del modelo de regresión

#### Creación de modelo predictivo de ventas por MES usando los datos de años anteriores ( lineal y polinomica ) Calculo R2

#### Creación de modelo predictivo de ventas por TRIMESTRE usando los datos de años anteriores ( lineal y polinomica ) Calculo R2

#### Creación de modelo predictivo de ventas por AÑO usando los datos de años anteriores ( lineal y polinomica ) Calculo R2

#### Modelos de regresión multiple entre todas las variables según  apartado 5.6. En especial, precio medio y relación con diferentes calibres.Precio vs Total Bags

#### Analisis de coeficientes de regresion multiple. Afectación de cada variable al AvgPrice apartado 5.7

#### Modelos de regresión para diferenciar volumenes de ventas Apartado 5.8 . Calibres y AveragePrice.

#### Creación de modelo de regresión y polinómica de ventas totales sobre precio promedio ( por regiones o no ) apartado 5.9

#### Predicciones

#### Predicción de precio promedio según volumen aguacates y por tipo ( y por calibres ??) . Lineal y Polinomica. Calculo de R2